# Smallville Valentine's Party — Miniverse Walkthrough

A micro-replication inspired by the Stanford Generative Agents “Valentine’s Party” experiment.

What you’ll see:
- Deterministic physics (time and basic updates) + LLM-driven agent cognition
- Prompt system defaults (system: identity + instructions + action catalog; user: initial state + perception)
- Information diffusion via communication memories (sender + recipient)

Run time: a few minutes (depends on LLM latency).


## Simulation Flow

1. Define world + deterministic physics (`SimulationRules`)
2. Define agents (`AgentProfile`) and initial locations (`AgentStatus`)
3. Provide action catalog + cognition stack (`LLMExecutor`/`LLMPlanner`)
4. Configure orchestrator, preview prompts, run for N ticks
5. Inspect actions, memories, and time progression


## Prerequisites

- Python 3.10+
- `uv` installed (for dependency management)
- Miniverse repo checked out
- LLM credentials configured via environment variables

Quickstart:
```bash
uv sync
export LLM_PROVIDER=openai
export LLM_MODEL=gpt-5-nano
export OPENAI_API_KEY=your_key
```



In [1]:
# Core imports
import os
from datetime import datetime, timezone

from miniverse import (
    Orchestrator, AgentProfile, AgentStatus, WorldState,
    ResourceState, EnvironmentState, SimulationRules, Stat
)
from miniverse.cognition import AgentCognition, LLMExecutor, LLMPlanner, Scratchpad
from miniverse.persistence import InMemoryPersistence
from miniverse.schemas import AgentPerception
from miniverse.cognition.prompts import DEFAULT_PROMPTS
from miniverse.cognition.context import PromptContext
from miniverse.cognition.renderers import render_prompt


In [2]:
# Time + physics rules
class TownSimulationRules(SimulationRules):
    """3-hour ticks with day rollover; simple activity stamping.
    """
    def __init__(self, *, tick_hours: int = 3, start_hour: int = 15, start_day: int = 13, month_label: str = 'Feb') -> None:
        self.tick_hours = tick_hours
        self.start_hour = start_hour
        self.start_day = start_day
        self.month_label = month_label

    def apply_tick(self, state: WorldState, tick: int) -> WorldState:
        updated = state.model_copy(deep=True)
        total_hours = self.start_hour + (tick * self.tick_hours)
        day_offset = total_hours // 24
        hour24 = total_hours % 24
        if hour24 == 0:
            hour12, ampm = 12, 'am'
        elif 1 <= hour24 < 12:
            hour12, ampm = hour24, 'am'
        elif hour24 == 12:
            hour12, ampm = 12, 'pm'
        else:
            hour12, ampm = hour24 - 12, 'pm'
        hr = updated.resources.get_metric('hour', default=hour12, unit=ampm, label='Current Time')
        hr.value, hr.unit = hour12, ampm
        day = updated.resources.get_metric('day', default=self.start_day, unit=self.month_label, label='Date')
        day.value, day.unit = self.start_day + day_offset, self.month_label
        updated.tick = tick
        return updated

    def validate_action(self, action, state):
        return True



In [3]:
# Build world + agents
world_state = WorldState(
    tick=0,
    timestamp=datetime.now(timezone.utc),
    environment=EnvironmentState(metrics={}),
    resources=ResourceState(metrics={
        'hour': Stat(value=3, unit='pm', label='Current Time'),
        'day': Stat(value=13, unit='Feb', label='Date')
    }),
    agents=[
        AgentStatus(agent_id='isabella', location='hobbs_cafe', display_name='Isabella Rodriguez'),
        AgentStatus(agent_id='maria', location='park', display_name='Maria Lopez'),
        AgentStatus(agent_id='klaus', location='park', display_name='Klaus Mueller'),
        AgentStatus(agent_id='ayesha', location='hobbs_cafe', display_name='Ayesha Khan'),
        AgentStatus(agent_id='tom', location='park', display_name='Tom Moreno'),
    ]
)

agents = {
    'isabella': AgentProfile(
        agent_id='isabella', name='Isabella Rodriguez', age=28,
        background="I opened Hobbs Cafe to bring people together.",
        role='cafe_owner', personality='warm and social',
        skills={'hospitality': 'expert', 'event_planning': 'expert', 'cooking': 'advanced'},
        goals=["Build a thriving community hub", "Host a Valentine's Day party at the cafe on Feb 14, 5-7pm"],
        relationships={'maria': 'Friend; knows she likes Klaus', 'klaus': 'Occasional customer'}
    ),
    'maria': AgentProfile(
        agent_id='maria', name='Maria Lopez', age=26,
        background='Graduate student who studies at the cafe.',
        role='student', personality='thoughtful and romantic',
        skills={'research': 'expert', 'writing': 'advanced'},
        goals=['Finish thesis', 'Reconnect with Klaus'],
        relationships={'isabella': 'Trusts her advice', 'klaus': 'Old friend'}
    ),
    'klaus': AgentProfile(
        agent_id='klaus', name='Klaus Mueller', age=27,
        background='Local musician focused on composing.',
        role='musician', personality='creative and introspective',
        skills={'music_composition': 'expert', 'piano': 'expert'},
        goals=['Finish album', 'Build local following'],
        relationships={'maria': 'Old friend', 'isabella': 'Friendly cafe owner'}
    ),
    'ayesha': AgentProfile(
        agent_id='ayesha', name='Ayesha Khan', age=30,
        background='Journalist covering local community stories.',
        role='journalist', personality='curious and observant',
        skills={'journalism': 'expert', 'writing': 'expert'},
        goals=['Cover meaningful local stories'],
        relationships={'isabella': 'Great source'}
    ),
    'tom': AgentProfile(
        agent_id='tom', name='Tom Moreno', age=32,
        background='Hardware store owner; reliable and helpful.',
        role='shopkeeper', personality='practical and straightforward',
        skills={'business': 'advanced', 'hardware_expertise': 'expert'},
        goals=['Support other local businesses'],
        relationships={'isabella': 'Mutual support between businesses'}
    ),
}


In [4]:
# Action catalog and cognition
available_actions = [
    {
        "name": "communicate",
        "schema": {
            "action_type": "communicate",
            "target": "agent_id",
            "parameters": None,
            "reasoning": "string",
            "communication": {"to": "agent_id", "message": "string"}
        },
        "examples": [{
            "action_type": "communicate",
            "target": "agent_b",
            "parameters": None,
            "reasoning": "Coordinate with another agent",
            "communication": {"to": "agent_b", "message": "Hello! Let's sync."}
        }],
    },
    {
        "name": "move_to",
        "schema": {"action_type": "move_to", "target": "location_id", "parameters": {}},
        "examples": [{"action_type": "move_to", "target": "location_alpha", "parameters": {}, "reasoning": "Move to a new location", "communication": None}],
    },
]

cognition_map = {
    agent_id: AgentCognition(
        executor=LLMExecutor(template_name="default", available_actions=available_actions),
        planner=LLMPlanner(),
        scratchpad=Scratchpad(),
    )
    for agent_id in agents.keys()
}

agent_prompts = {
    'isabella': "It's February 13th. You're excited about hosting a Valentine's Day party tomorrow (Feb 14, 5-7pm). You want it warm and inclusive. Think about who might enjoy coming.",
    'maria': "You've been spending long days at the cafe working on your thesis. Isabella has become a good friend.",
    'klaus': "You've been in a creative flow composing music; you sometimes forget to socialize.",
    'ayesha': "You're always on the lookout for community stories that bring neighborhoods together.",
    'tom': "Business is steady. You support other local business owners when you can.",
}


In [5]:
# Orchestrator + preflight prompt view
rules = TownSimulationRules(tick_hours=3, start_hour=15, start_day=13)

orchestrator = Orchestrator(
    world_state=world_state,
    agents=agents,
    world_prompt='This is smallville, a quiet town with a few local businesses and a community cafe called Hobbs Cafe.',
    agent_prompts=agent_prompts,
    simulation_rules=rules,
    agent_cognition=cognition_map,
    llm_provider=os.getenv('LLM_PROVIDER', 'openai'),
    llm_model=os.getenv('LLM_MODEL', 'gpt-5-nano'),
    persistence=InMemoryPersistence(),
    memory=None,
    world_update_mode='deterministic'
)

# Show prompts at tick 0 for transparency
for agent_id, profile in agents.items():
    context = PromptContext(
        agent_profile=profile,
        perception=AgentPerception(agent_id=agent_id, tick=0),
        world_snapshot=world_state,
        scratchpad_state={},
        plan_state={},
        memories=[],
        extra={
            "initial_state_agent_prompt": agent_prompts.get(agent_id, ""),
            "available_actions": available_actions,
            "simulation_instructions": "You are an agent in a simulation. Read perception and return an AgentAction JSON. Use only the available actions.",
        },
    )
    tmpl = DEFAULT_PROMPTS.get("default")
    rendered = render_prompt(tmpl, context, include_default=False)
    print("\n" + "━"*80)
    print(f"AGENT: {profile.name}")
    print("━"*80)
    print("\n[SYSTEM]")
    print(rendered.system)
    print("\n[USER]")
    print(rendered.user)



━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
AGENT: Isabella Rodriguez
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[SYSTEM]
I am Isabella Rodriguez.
I am 28 years old.
I work as a cafe owner.
Background: I opened Hobbs Cafe to bring people together.
My personality is warm and social.
My skills include: hospitality (expert), event_planning (expert), cooking (advanced).
My goals are: Build a thriving community hub, Host a Valentine's Day party at the cafe on Feb 14, 5-7pm.
My relationships with others:
- maria: Friend; knows she likes Klaus
- klaus: Occasional customer

You are an agent in a simulation. Read perception and return an AgentAction JSON. Use only the available actions.

Available actions:
Action Catalog (choose one):
- communicate
  Schema:
    {
      "action_type": "communicate",
      "target": "agent_id",
      "parameters": null,
      "reasoning": "string",
      "communication": {
        "to":

In [6]:
# Notebook-safe run
import asyncio, nest_asyncio
nest_asyncio.apply()

async def run():
    return await orchestrator.run(num_ticks=10)

try:
    # In Jupyter with an active loop
    _ = asyncio.get_running_loop()
    result = await run()
except RuntimeError:
    # No active loop (script-like)
    result = asyncio.run(run())

print("\n✅ Simulation complete!")

Starting simulation run 8ca88c65-97cb-4e19-801a-37541d900dd6
Agents: 5, Ticks: 10

  [Prompts] Agent 'isabella' planner using default template 'plan'.
  [Prompts] Agent 'isabella' executor using default template 'default'.
  [Prompts] Agent 'maria' planner using default template 'plan'.
  [Prompts] Agent 'maria' executor using default template 'default'.
  [Prompts] Agent 'klaus' planner using default template 'plan'.
  [Prompts] Agent 'klaus' executor using default template 'default'.
  [Prompts] Agent 'ayesha' planner using default template 'plan'.
  [Prompts] Agent 'ayesha' executor using default template 'default'.
  [Prompts] Agent 'tom' planner using default template 'plan'.
  [Prompts] Agent 'tom' executor using default template 'default'.
[Preflight] World updates: deterministic (basic)
=== Tick 1/10 ===
  [•] [Physics] Applying deterministic rules for tick 1...
  [✓] [Physics] Physics applied
  [•] [Isabella Rodriguez] Building perception...
  [•] [Maria Lopez] Building percep